In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import csv
import os 
import scipy.stats as st

ModuleNotFoundError: No module named 'gurobipy'

### Model prepared for Objective 3 
<br>
Stochastic parameter of yiled 
<br>
2 years model 


In [ ]:
import time
start = time.process_time()

In [ ]:
os.chdir("C:/Coursework/Dissertation/Ch3/Data/Y15A01")

### Input Data, sets and paramters

In [ ]:
alpha = 0.01
Z_alpha =  st.norm.ppf(1-alpha) 
print(Z_alpha)
(1+0.019)/(1+0.06)

In [ ]:
#tt = 0.907185819  # ton to t conversion  
#reading county list]
CountyData = csv.reader(open('County.csv'))
c = []
for i in CountyData:
    c.append(i[0])

d = ['d']    #Demand node 

t=15 # Setting simulation years
t = [*range(1,t+1)]  # set of time 
q = 4  # number of quarters in each year 
q = [*range(1,q+1)] # setting set of quarter 
t_q = [(i,j) for i in t for j in q] # Setting set of each quarter in each year 

#Discounted value
beta = 0.96  # numerator is inflation, denominator is discounted value. 
beta_tq = {(i,j):beta**(i-1) for i in t for j in q}  # setting set of beta for each quarter-year

PP = [1,0,0,0]*len(t)      # Production potential 
PP_tq = dict(zip(t_q, PP))  #Production potential at t,q        
 
#Seed Supply 
readArea = csv.reader(open('Area.csv')) #Area for county c 
next(readArea) #skips the headers
A = {}  
for row in readArea:
    A[row[0]]=float(row[1])

readYeild = csv.reader(open('YieldMeanSTD.csv')) #Yeild data
next(readYeild) #skips the headers

#Z_alpha =  st.norm.ppf(1-alpha)  # set the z-score under given alpha 

S = {}    #setting set of supply
for row in readYeild:
    S[row[0]]=float(row[2]) - 2.32*float(row[3])  # row[2] is mean, row[3] is sd of yields 

seed = {(i,j,k): S[i]*PP_tq[j,k] for i in c for j,k in PP_tq} #potential seed supply for each quarter-year

#Y = {i: seed[i,1,1]/(A[i]/3) for i in c}
storVol = 1134 #Volume of storages 
storage = {(i,j,k): storVol for i in c for j,k, in PP_tq}   
openStorecost = 125000 # {(i,j,k): 125000*0.9*2/20  for i in c for j,k, in PP_tq}

crushVol = 1037323 # Volume of crushing mills #495000/0.4329
crush = {(i,j,k): crushVol for i in c for j,k, in PP_tq} 
openCrushcost = 16130000 #{(i,j,k): 16130000*0.8*2/20  for i in R for j,k, in PP_tq}

refineVol =  110279 # Volume of biorefinary #87500/0.7198
refinary = {(i,j,k): refineVol for i in c for j,k, in PP_tq} 
openRefinarycost = 409220000 #{(i,j,k): 409220000*0.8*2/20 for i in R for j,k, in PP_tq}

#Transport data inputs 
readTC_fs = csv.reader(open('SeedOD.csv')) # read transport cost data for field to storages, field to crushing mills, 
                                            #Storage to crushing mills 
TC_fs = {}
for row in readTC_fs:
    key1=row[0]
    key2=row[1]
    TC_fs[(key1,key2)]=float(row[2])
TC1 = {(i,j,k,m): TC_fs[i,j] * beta_tq[k,m] for i in c for j in c for k,m in PP_tq}
arcs1, cost1 = gp.multidict(TC1)

readTC_c = csv.reader(open('OilOD.csv')) # read transport cost data for crushing mill to bioerfinary 
TC_c = {}
for row in readTC_c:
    key1=row[0]
    key2=row[1]
    TC_c[(key1,key2)]=float(row[2])
TC2 = {(i,j,k,m): TC_c[i,j] * beta_tq[k,m] for i in c for j in c for k,m in PP_tq}
arcs2, cost2 = gp.multidict(TC2)

readTC_b = csv.reader(open('SAF_OD.csv')) # read transport cost data for bioerfinary to airport
TC_b = {}
for row in readTC_b:
    key1=row[0]
    key2=row[1]
    TC_b[(key1,key2)]=float(row[2])
TC3 = {(i,j,k,m): TC_b[i,j] * beta_tq[k,m] for i in c for j in d for k,m in PP_tq}
arcs3, cost3 = gp.multidict(TC3)

#Operation cost
LU = 4.4  #4 / tt
SOP = 8.8 #8 / tt
OOP = 19.54 #17.73 / tt
BOP = 941.2 #853.9 / tt

P = 441 #400/tt  # Price of carinata $/t
#Coproduct Prices 
MP = 413.3 #375 / tt #Price of carinata meal $/ton 
PPr = 670.2 # 608 / tt # Price of propen $/ton
NP = 1030.7 #935 / tt # Price of naphtha $/ton

#Factors 
O_prime = 0.4329 # The conversion factor from seeds to oil
J = 0.7198  # The conversion factor from oil to bio-jet fuel
M = 0.56 # The cnversion factor from seed to meal 
Pr = .0882 # Conversion factor of Propen from oil
N = .062 # Conversion factor of Naphtha frm oil 

AvgDemand = 879533 #969518.12*tt  # Average quarterly demand of total jet fuel at the airport d, ton/quarter 
#D_prime = 0.106 # Share of total jet fuel demand, that must be met by bio-jet fuel
#Demand = {(i,j): AvgDemand*D_prime for i in t for j in q}
SL = 0.99

#DemandQ = sum(seed[i,1,1] for i in c)*SL**3*1/(1+SL+SL**2+SL**3)*O_prime*J # total demand that can be met by bio-jet fuel
DemandQ = AvgDemand*.096
Demand = {(i,j): DemandQ for i,j in PP_tq}
#print(DemandQ/AvgDemand, AvgDemand*.096)

#print(sum(seed[i,1,1] for i in c)*O_prime*J, sum(Demand[1,1]))
#print(Y)

Setting decision variables 

In [ ]:
model = gp.Model('SupplyNetworkDesign')

#Flow contraints linear program 
supply = model.addVars(seed, name="supply")
flow1 = model.addVars(arcs1, obj=cost1, name="flow1") #Field to store
flow2 = model.addVars(arcs1, obj=cost1, name="flow2") #Field to crush
flow3 = model.addVars(arcs1, obj=cost1, name="flow3") #Store to crush
flow4 = model.addVars(arcs2, obj=cost2, name="flow4") #Crush to biorefinary
flow5 = model.addVars(arcs3, obj=cost3, name="flow5") # Biorefinary to airport
stock = model.addVars(storage, name="stock")

#Facilities constraints integer program 
openStore = model.addVars(c, obj=openStorecost, vtype=GRB.INTEGER, name="openStore")
openCrush = model.addVars(c, obj=openCrushcost, vtype=GRB.BINARY, name="openCrush")
openRefinary = model.addVars(c, obj=openRefinarycost, vtype=GRB.BINARY, name="openRefinary")

Setting Constraints

In [ ]:
# Facility capacity limits
stores = storage.keys()
crushes = crush.keys()
refine = refinary.keys()
# Materials Flow

farms = seed.keys()
farm_supply = model.addConstrs(supply[farm,j,k] <= seed[farm,j,k] for farm,j,k in farms)
#crush_volume1 = model.addConstrs((gp.quicksum(flow1.select('*', 'crush', i, j)) <= openCrush[crush, i,j]* demand[crush, i,j]
                               # for crush,i,j in crushes), name="crush1")
farm_store = model.addConstrs((gp.quicksum(flow1.select(farm, '*', i, j)) + gp.quicksum(flow2.select(farm, '*', i, j))<= 
                               supply[farm, i, j] for farm,i,j in farms), name="farmStore")   

Stock = model.addConstrs((stock[i,j,k] == gp.quicksum(flow1.select('*', i, j, k)) - gp.quicksum(flow3.select(i, '*', j, k)) 
                          for i in c for j in t for k in q if k==1), name="Stock")
Stock = model.addConstrs((stock[i,j,k] == stock[i,j,k-1]*SL -
                          gp.quicksum(flow3.select(i, '*', j, k)) for i in c for j in t for k in q if k>1), name="Stock")

farm_store = model.addConstrs((gp.quicksum(flow1.select('*', store, i, j)) <= openStore[store]* storage[store,i,j]
                                 for store,i,j in stores), name="farmStore")

crush_volume = model.addConstrs((gp.quicksum(flow2.select('*', crushMill, i, j))+ 
                                 gp.quicksum(flow3.select('*', crushMill, i, j)) <= openCrush[crushMill]*crush[crushMill,i,j]
                                 for crushMill,i,j in crushes), name="crushVol")

oil_volume = model.addConstrs((gp.quicksum(flow4.select(crushMill,'*', i,j)) == (gp.quicksum(flow3.select('*', crushMill, i, j))+
                                                                           gp.quicksum(flow2.select('*', crushMill, i, j)))*O_prime
                               for crushMill, i,j in crushes), name="oil_volume")

refine_volume = model.addConstrs((gp.quicksum(flow4.select('*',biorefine, i,j)) <= openRefinary[biorefine]*refinary[biorefine,i,j]
                                 for biorefine,i,j in refine), name='refine_volume')

SAF_flow = model.addConstrs((gp.quicksum(flow5.select(biorefine,d,i,j)) == gp.quicksum(flow4.select('*',biorefine,i,j))*J
                             for biorefine, i, j in refine), name="SAF_flow")

Total_volume = model.addConstrs((gp.quicksum(flow5.select('*', '*', i, j)) >= Demand[i,j]
                              for i in t for j in q), name="TotalDemand")

In [ ]:
#store_count =model.addConstr(openStore.sum() >= 991)
crush_count = model.addConstr(openCrush.sum() >=1)
refinary_count = model.addConstr(openRefinary.sum() >= 1)

Objective Function 

In [ ]:
SeedCost = sum(P * supply[farms, i,j] * beta_tq[i,j] for farms in c for i in t for j in q)

#Capital Cost 
CapitalCost1 = openStore.sum()* openStorecost
CapitalCost2 = openCrush.sum()* openCrushcost
CapitalCost3 = openRefinary.sum()* openRefinarycost
CapitalCost = CapitalCost1 + CapitalCost2 + CapitalCost3

#Operational Cost 

FSxLUxbeta = {(c1,c2,i,j): flow1[c1,c2,i,j]*LU*beta_tq[i,j] for c1,c2,i,j in flow1}
SOxLUxbeta = {(c1,c2,i,j): flow3[c1,c2,i,j]*LU*beta_tq[i,j] for c1,c2,i,j in flow3}

OBxOOPxbeta = {(c1,c2,i,j): flow4[c1,c2,i,j]*OOP*beta_tq[i,j] for c1,c2,i,j in flow4}
BAxBOPxbeta = {(c1,c2,i,j): flow5[c1,c2,i,j]*BOP*beta_tq[i,j] for c1,c2,i,j in flow5}

OP1 = sum(FSxLUxbeta[i,j,k,m] for i,j,k,m in flow1)
OP2 = sum(SOxLUxbeta[i,j,k,m] for i,j,k,m in flow3)

OP3 = sum(stock[store, i,j] * SOP * beta_tq[i,j] for store in c for i in t for j in q)
OP4 = sum(OBxOOPxbeta[i,j,k,m] for i,j,k,m in flow4)
OP5 = sum(BAxBOPxbeta[i,j,k,m] for i,j,k,m in flow5)

OperationCost = OP1+OP2+OP3+OP4+OP5 

#Transport Cost 
TCxFSxbeta = {(c1,c2,i,j): TC1[c1,c2,i,j]*flow1[c1,c2,i,j] for c1,c2,i,j in flow1}
TC_a = sum(TCxFSxbeta[i,j,k,m] for i,j,k,m in flow1)

TCxFOxbeta = {(c1,c2,i,j): TC1[c1,c2,i,j]*flow2[c1,c2,i,j] for c1,c2,i,j in flow2}
TC_b = sum(TCxFOxbeta[i,j,k,m] for i,j,k,m in flow2)

TCxSOxbeta = {(c1,c2,i,j): TC1[c1,c2,i,j]*flow3[c1,c2,i,j] for c1,c2,i,j in flow3}
TC_c = sum(TCxSOxbeta[i,j,k,m] for i,j,k,m in flow3)

TCxOBxbeta = {(c1,c2,i,j): TC2[c1,c2,i,j]*flow4[c1,c2,i,j] for c1,c2,i,j in flow4}
TC_d = sum(TCxOBxbeta[i,j,k,m] for i,j,k,m in flow4)

TCxBAxbeta = {(c1,c2,i,j): TC3[c1,c2,i,j]*flow5[c1,c2,i,j] for c1,c2,i,j in flow5}
TC_e = sum(TCxBAxbeta[i,j,k,m] for i,j,k,m in flow5)

TransportCost = TC_a+TC_b+TC_c+TC_d+TC_e 

#Coproduct Cost 
FOxMxMPxbeta = {(c1,c2,i,j): flow2[c1,c2,i,j]*M*MP*beta_tq[i,j] for c1,c2,i,j in flow2}
Cop1 = sum(FOxMxMPxbeta[i,j,k,m] for i,j,k,m in flow2)

SOxMxMPxbeta = {(c1,c2,i,j): flow3[c1,c2,i,j]*M*MP*beta_tq[i,j] for c1,c2,i,j in flow3}
Cop2 = sum(SOxMxMPxbeta[i,j,k,m] for i,j,k,m in flow3)

OBxNxNPxbeta = {(c1,c2,i,j): flow4[c1,c2,i,j]*N*NP*beta_tq[i,j] for c1,c2,i,j in flow4}
Cop3 = sum(OBxNxNPxbeta[i,j,k,m] for i,j,k,m in flow4)

OBxPrxPPrxbeta = {(c1,c2,i,j): flow4[c1,c2,i,j]*Pr*PPr*beta_tq[i,j] for c1,c2,i,j in flow4}
Cop4 = sum(OBxPrxPPrxbeta[i,j,k,m] for i,j,k,m in flow4)

CoproductSale = Cop1+Cop2+Cop3+Cop4

model.setObjective(SeedCost + CapitalCost + OperationCost + TransportCost - CoproductSale) 

#first_six = list(Stock)[:12]
print("Done!")

In [ ]:
model.setParam('MIPGap', 0.0125)
#model.setParam('Timelimit', 60*60*2)
model.optimize()

print("Done!")

In [ ]:
print("Cost (in $):", "\n", 
     "Seed Cost:", SeedCost.getValue(), "\n",                    
      "Capital Cost:", CapitalCost.getValue(), "\n",
      "Loading:", OP1.getValue(), "\n",
      "Storage holding:", OP3.getValue(), "\n",
      "Unloading:", OP2.getValue(), "\n",
      "OEM:", OP4.getValue(), "\n",
      "Bio:", OP5.getValue(), "\n",
      "Transportation Cost:", TransportCost.getValue(), "\n",
     "Coproduct Sale:", CoproductSale.getValue(), "\n",
     "Total Cost:", SeedCost.getValue()+CapitalCost.getValue()+OperationCost.getValue()+TransportCost.getValue()-CoproductSale.getValue(), "\n",
      "Total SAF Demand Per Quarter:", Demand[1,1], "\n",
     "Demand Ratio of SAF to CAF:", Demand[1,1]/AvgDemand)

#Exporting results 

In [ ]:
flow_one={}
for i in c:
    for j in c:
        for k in t:
            for m in q:
                if flow1[(i,j,k,m)].x > 0.001:
                    flow_one[(i,j,k,m)]=flow1[(i,j,k,m)].x
pd.DataFrame(flow_one, index=[0]).to_csv('FieldToStore2GIS.csv')

flow_two={}
for i in c:
    for j in c:
        for k in t:
            for m in q:
                if flow2[(i,j,k,m)].x > 0.001:
                    flow_two[(i,j,k,m)]=flow2[(i,j,k,m)].x
pd.DataFrame(flow_two, index=[0]).to_csv('FieldToCrush2GIS.csv')

flow_three={}
for i in c:
    for j in c:
        for k in t:
            for m in q:
                if flow3[(i,j,k,m)].x > 0.001:
                    flow_three[(i,j,k,m)]=flow3[(i,j,k,m)].x
pd.DataFrame(flow_three, index=[0]).to_csv('StorToCrush2GIS.csv')

flow_four={}
for i in c:
    for j in c:
        for k in t:
            for m in q:
                if flow4[(i,j,k,m)].x > 0.001:
                    flow_four[(i,j,k,m)]=flow4[(i,j,k,m)].x
pd.DataFrame(flow_four, index=[0]).to_csv('CrushToBiorefine2GIS.csv')

flow_five={}
for i in c:
    for j in d:
        for k in t:
            for m in q:
                if flow5[(i,j,k,m)].x > 0.001:
                    flow_five[(i,j,k,m)]=flow5[(i,j,k,m)].x
pd.DataFrame(flow_five, index=[0]).to_csv('BioRefineToAirport2GIS.csv')

StoreLoc = {}
for i in c:
    if openStore[i].x > 0.5:
        StoreLoc[i] = openStore[i].x
pd.DataFrame(StoreLoc, index=[0]).to_csv('StoreLoc2GIS.csv')

CrushLoc = {}
for i in c:
    if openCrush[i].x > 0.5:
        CrushLoc[i] = openCrush[i].x
pd.DataFrame(CrushLoc, index=[0]).to_csv('CrushLoc2GIS.csv')

BiorefineLoc = {}
for i in c:
    if openRefinary[i].x > 0.5:
        BiorefineLoc[i] = openRefinary[i].x
pd.DataFrame(BiorefineLoc, index=[0]).to_csv('BiorefineLoc2GIS.csv')

Stock = {}
for i in c:
    for j in t:
        for k in q:
            if stock[i,j,k].x >.001:
                Stock[i,j,k] = stock[i,j,k].x
pd.DataFrame(Stock, index=[0]).to_csv('Stock_2GIS.csv')

Seed = {}
for i in c:
    for j in t:
        for k in q:
            if supply[i,j,k].x >.001:
                Seed[i,j,k] = supply[i,j,k].x
pd.DataFrame(Seed, index=[0]).to_csv('Seed_2GIS.csv')

Harvest= {}
for i in c:
    for j in t:
        for k in q:
            if supply[i,j,k].x >.001:
                Harvest[i,j,k] = supply[i,j,k].x/Y[i]
pd.DataFrame(Harvest, index=[0]).to_csv('Harvest_2GIS.csv')

In [ ]:
print((time.process_time() - start)/(60*60))
#after seting objective function (adding all costs): 0.6340190972222223

In [ ]:
print(969518.12*0.907185819*.13, 110279) #.096